本文内容主要来自：
- https://arxiv.org/pdf/1704.04347.pdf

## 文章简介
在进行翻译的时候，如果能够考虑到整篇文章的信息，可以帮助我们出来可能出现的二义性问题。在这篇文章中，我们提出一种方法，他可以利用历史信息（即之前几个句子的信息）来帮助当前句子的翻译。<br>
在这里，历史信息是使用分层结构进行存储的。我们提出两种把这个结果整合到NMT的方法：
- warm-start of encoder and decoder states
- an auxiliary context source for updating decoder states.

实验证明，这个方法在中译英的基于Attention的NMT系统中，可以提高2.1BLEU

## 介绍
最近一段时间，NMT的发展很快。并取得了很多令人瞩目的成绩。但是他在一下几方面还是存在不足:
- ？？the continuous vector representation, or a word embedding vector, of a symbol encodes multiple dimensions of similarity, equivalent to encoding more than one meaning of the word. This has the consequence that the encoder and decoder recurrent networks in neural machine translation need to spend substantial amount of their capacity in disambiguating source and target words based on the context which is defined by a source sentence. 
- 另外，在进行文章层面上的翻译的时候，一致性问题显得尤为重要。也就是说相同的单词在文章的不同的位置，应该有相同的翻译。
- 在对文章进行翻译的时候，目前的NMT系统还是维持各个句子单独翻译的模式。

cross-sentence context在一下几方面，被证明是存在正面意义的:
- capture the meaning or intention in sequential tasks such as query suggestion 
- dialogue modeling

在这篇文章中，我们提出了一个新的模型，cross-sentence context-aware NMT model， 他在对一个句子进行翻译的时候，会考虑在这篇文章中，前面的句子的内容。<br>
具体来说，我们使用一个分层的RNN来总结原文方面，历史句子的上下文内容。这个分层的RNN由两部分组中，即sentence-level的RNN encoder以及位于其上的document-level的RNN。在获得global context之后，我们设计了一些策略，使得这些信息能够融入到NMT中，用于翻译当前的句子:
- 初始化，使用代表历史信息的向量，来出事后encoder或者decoder，或者这两个家伙。
- 作为辅助的context内容（Auxiliary Context）。把获得的历史信息做为静态的跨越句子的上下文，并把他和由Attention机制输出得到的动态的上下文进行结合
- gating Auxiliary Context, 就是在上面提到的Auxiliary Context上计入一个gate。

## 具体方法
给定要求翻译的句子$x_m$，我们考虑这个句子之前的相同文章中的k个句子作为其上下文C= {$x_{m-k},...,x_{m-1}$}
### Summarizing Global Context
![cross senstence context](https://raw.githubusercontent.com/HuangYiran/readPaper/master/fotos/cross_sent.png)
如图所示，我们使用分层结构来归纳上下文信息。
- Sentence-lvel RNN: 针对C中每个句子$x_k$, 我们把其中的每个单词，依照顺序读入RNN中，并在处理每个单词后得到其隐藏层输出，使得$h_{n,k}=f(h_{n-1,k},x_{n,k})$。其中$h_{n,k}$表示在时间点n的RNN的隐藏层的信息。最后的输出状态$h_{N,k}$包含了句子中各个单词的信息（包括其顺序），我们用这个状态来代表这个句子。
- Document-level RNN：使用上面sentence-level RNN的输出做为输入，其操作和sentence-level RNN相同。并把最终的隐藏层输出D作为我们想要的global context

### Integrating Global Context into NMT
我们提供下面几个策略来结合得到的上下文信息
##### Initialization
我们使用D来初始化NMT的encoder或decoder<br>
- 对于encoder来说，我们使用D作为初始状态（一般$h_{-1}$初始化为全0）。
- 对于decoder来说，我们对初始状态作一些调整。一般情况下$s_0 = tanh(W_sh_N)$，其中$h_N$代表encoder的最终状态。我们把他调整为$s_0 = tanh(W_sh_N+W_DD)$

##### Auxiliary Context && Gating Auxiliary Context
![cross sentence](https://raw.githubusercontent.com/HuangYiran/readPaper/master/fotos/cross_sent2.png)


## 下面几个句子表示不理解
1-2: The continuous vector representation of a symbol encodes multiple dimensions of similarity, equivalent to encoding more than one meaning of a word？？？<br>
2-1： sentence RNN的几个表达感觉有点疑惑，不用在意RNN的参数吗？？？？